# Analysing 1D SWE model outputs across a variety of different parameters

Recall the SWE:

\begin{align*}
    \partial_t u + u \cdot \nabla u + g \nabla \eta - \nu \nabla^2 u &= 0, \\
    \partial_t \eta + \nabla \cdot ((\eta + \eta) u) &= 0.
\end{align*}

For our paper simulation study -- and thus, also in this notebook -- we are looking at analysing the results of running statFEM for the SWE, with different viscosity parameters $\nu$, topographies $H$, and observation intervals. We will mainly analyse outputs in terms of the relative RMSE:

\begin{equation*}
    \mathrm{RMSE} = \frac{\Vert y_m - H(u_m, h_m)\Vert}{\Vert y_m \Vert},
\end{equation*}

and also the (negative) log-likelihoods for a probabilistic metric for comparison.

## Initialising data and getting things set up

Here we first define some helper functions and the like, to be used later on. We also import libraries and some definitions for the figure sizes:

In [1]:
import h5py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

from tabulate import tabulate

import sys
sys.path.insert(0, "../scripts")
from figures import figure_sizes

def errors_from_output_file(output_file, prior=False):
    """ Read in errors from output file. """
    checkpoint = h5py.File(output_file, "r")
    nu = checkpoint["/"].attrs["nu"]
    c = checkpoint["/"].attrs["c"]
    nt_skip = checkpoint["/"].attrs["nt_skip"]
    k = checkpoint["/"].attrs["k"]
    linear = checkpoint["/"].attrs["linear"]
    
    # just verify we are doing the correct thing
    assert checkpoint["/"].attrs["rho_h"] == 1e-2 
    t_obs = checkpoint["/t_obs"][:]
    rmse = checkpoint["/rmse"][:]
    rmse_rel = checkpoint["/rmse_rel"][:]
    
    # common coordinates across all
    coords=dict(t=t_obs,
                nu=np.array([nu]),
                c=np.array([c]),
                nt_skip=np.array([nt_skip]))
    
    rmse_ds = xr.DataArray(data=rmse[:, np.newaxis, np.newaxis, np.newaxis],
                           coords=coords,
                           name="rmse")
    rmse_rel_ds = xr.DataArray(data=rmse_rel[:, np.newaxis, np.newaxis, np.newaxis],
                               coords=coords,
                               name="rmse_rel")
    if not prior:
        lml = checkpoint["/lml"][:]
        checkpoint.close()
        
        lml_ds = xr.DataArray(data=lml[:, np.newaxis, np.newaxis, np.newaxis],
                              coords=coords,
                              name="lml")
        return (nu, nt_skip), xr.merge([rmse_ds, rmse_rel_ds, lml_ds])
    else:
        checkpoint.close()
        return (nu, nt_skip), xr.merge([rmse_ds, rmse_rel_ds])

First we load in the posterior model outputs and see what they look like:

In [2]:
from glob import glob

output_files = glob(
    "../outputs/swe-bump/*-posterior-c-*-nt_skip-*.h5")

errors_ds = []
for output_file in output_files:
    _, e = errors_from_output_file(output_file)
    errors_ds.append(e)

errors_nonlinear = xr.merge(errors_ds)
errors_nonlinear

KeyboardInterrupt: 

Next, we load in the prior model outputs, for `nt_skip = 100`, and check that they look OK:

In [ ]:
output_files_prior = glob(
    "../outputs/swe-bump/*-prior-c-*-nt_skip-100-*.h5")

errors_ds = []
for output_file in output_files_prior:
    _, e = errors_from_output_file(output_file, prior=True)
    errors_ds.append(e)

errors_prior = xr.merge(errors_ds)
errors_prior

Now we filter the output information to that which we are actually interested.

In [ ]:
c_subset = [0, 2, 5, 6]
errors_prior = errors_prior.isel(c=c_subset)
errors_nonlinear = errors_nonlinear.isel(c=c_subset)

## Paper plots: checking performance across observation intervals

First we plot the results across the different observation intervals to see how things stack up across time. We first look at the relative errors across time, for different values of `nt_skip`, for the true value of $c$:

In [ ]:
assert errors_nonlinear.coords["c"][1] == 10.

nus =  errors_nonlinear.coords["nu"].to_numpy()
labels = [
    fr"$\nu = ${nu:.1e}" if nu > 0 else "Linear" for nu in nus
]

p = errors_nonlinear.rmse_rel.isel(c=1, nu=slice(0, 6)).interpolate_na(dim="t").plot(
    hue="nt_skip", col="nu", col_wrap=3,
    figsize=(figure_sizes["fullwidth"], 0.6 * figure_sizes["fullwidth"]),
    sharex=True, sharey="row")

# figure customization
p.figlegend.set_title(r"$k$")
for i, ax in enumerate(p.axes.flat):
    ax.set_title(labels[i])

for ax in p.axes[1, :]:
    ax.set_xlabel(r"$t$")

for ax in p.axes[:, 0]:
    ax.set_ylabel("RRMSE") 
    
plt.savefig("../figures/swe-bump-paper/nt-skip-nu-grid.pdf", dpi=600)

And, again, for the relative errors for different $c$ values for the linear model:

In [ ]:
p = errors_nonlinear.rmse_rel.isel(nu=1).interpolate_na(dim="t").plot(
    hue="nt_skip", col="c", col_wrap=4,
    figsize=(figure_sizes["fullwidth"], 0.28 * figure_sizes["fullwidth"]),
    sharey=True)

# figure customization
# p.figlegend.set_title(r"$k$")
p.figlegend.remove()

cs =  errors_nonlinear.coords["c"].to_numpy()
labels = [fr"$c = {c:.0f}$" for c in cs]
for i, ax in enumerate(p.axes.flat):
    ax.set_title(labels[i])

for ax in p.axes.flat:
    ax.set_xlabel(r"$t$")

p.axes[0, 0].set_ylabel("RRMSE") 
plt.savefig("../figures/swe-bump-paper/nt-skip-c-grid.pdf", dpi=600)

In [ ]:
errors_nonlinear.rmse_rel.isel(nu=1).interpolate_na(dim="t").plot(
    hue="nt_skip", col="c", sharey=True)
plt.show()

The results are decent across the the board with minor stratifications between the models across for different `nt_skip` values, but with greater stratification from changing parameter values. This leads to the conclusion that the different time-differences between observations does not drive the variation in model performance, in this scenario. We now tabulate these results, highlighting the results with the lowester RRMSE:

In [ ]:
ave_rmse = errors_nonlinear.isel(c=1).mean(dim="t", skipna=True).rmse_rel
dat = ave_rmse.to_numpy().T

from scipy.stats import f_oneway

print(f_oneway(*[col for col in dat.T]))
print(tabulate(dat,
               ave_rmse.coords["nu"].to_numpy(),
               showindex=ave_rmse["nt_skip"].to_numpy()))
                 
def format_col(col, exp=False):
    min_loc = col.argmin()
    out = col.astype(str)
    
    for i in range(len(col)):
        if i == min_loc:
            if exp:
                out[i] = fr"$\mathbf{{{col[i]:.6e}}}$"
            else:
                out[i] = fr"$\mathbf{{{col[i]:.6f}}}$"
        else:
            if exp:
                out[i] = fr"${col[i]:.6e}$"
            else:
                out[i] = fr"${col[i]:.6f}$"

    return np.array(out)
            
dat_formatted = np.apply_along_axis(format_col, 0, dat)

with open("../figures/swe-bump-paper/nt-skip-nu-table.tex", "w") as f:
    f.write(tabulate(dat_formatted,
                     ave_rmse.coords["nu"].to_numpy(),
                     showindex=ave_rmse["nt_skip"].to_numpy(),
                     tablefmt="latex_raw"))

In [ ]:
ave_rmse = errors_nonlinear.isel(nu=1).mean(dim="t", skipna=True).rmse_rel

dat = ave_rmse.to_numpy().T
print(tabulate(dat, 
               ave_rmse.coords["c"].to_numpy(),
               showindex=ave_rmse["nt_skip"].to_numpy()))

print(f_oneway(*[col for col in dat.T]))
dat_formatted = np.apply_along_axis(format_col, 0, dat)

with open("../figures/swe-bump-paper/nt-skip-c-table.tex", "w") as f:
    f.write(tabulate(dat_formatted, 
                     ave_rmse.coords["c"].to_numpy(),
                     showindex=ave_rmse["nt_skip"].to_numpy(),
                     tablefmt="latex_raw"))

## Paper plots: checking performance for misspecified model parameters, with a fixed observation interval

First we plot the RRMSE values across the grid of $\nu$ and $c$ values, both a priori and a posteriori. Across different values this stratifies and results in reasonable results. As previous we see that at $\nu = 1$ the dynamics seems to change and is much less amenable to being updated with data, due to the damping -- perhaps there is a dynamical bifurcation here?

In [ ]:
plt.rcParams['figure.constrained_layout.use'] = True

errors_temp = xr.concat([errors_prior, errors_nonlinear], 
                        pd.Index(["Prior", "Posterior"], name="Model"))

p = errors_temp.rmse_rel.interpolate_na(dim="t").isel(nt_skip=4, nu=slice(0, 5)).plot(
    x="t", col="nu", row="Model", hue="c", 
    figsize=(1.5 * figure_sizes["fullwidth"], 0.5 * figure_sizes["fullwidth"]),
    sharex=True, sharey=True)

nus =  errors_temp.coords["nu"].to_numpy()[slice(0, 5)]
labels = [fr"$\nu =$ {nu:.1e}" for nu in nus]
labels[0] = "Linear"

for i, ax in enumerate(p.axes[0, :]):
    ax.set_title(labels[i])

for ax in p.axes[1, :]:
    ax.set_xlabel(r"$t$")
    
for ax in p.axes[:, 0]:
    ax.set_ylabel("RRMSE")
    
p.figlegend.set_title("$c$")

plt.savefig("../figures/swe-bump-paper/nu-c-rrmse-layout.pdf", dpi=600)

Now, we plot the results across the same grid, however this time we plot the log-marginal likelihoods:

In [ ]:
p = (-errors_nonlinear.lml).interpolate_na(dim="t").isel(nt_skip=4, nu=slice(0, 5)).plot(
    x="t", col="nu", hue="c", yscale="log",
    figsize=(1.5 * figure_sizes["fullwidth"], 0.3 * figure_sizes["fullwidth"]),
    sharex=True, sharey=True)

nus =  errors_temp.coords["nu"].to_numpy()[slice(0, 5)]
labels = [fr"$\nu =$ {nu:.1e}" for nu in nus]
labels[0] = "Linear"
p.axes[0, 0].set_ylabel(r"-$\log p(\mathbf{y}_{1:m})$")

for i, ax in enumerate(p.axes[0, :]):
    ax.set_title(labels[i])

for ax in p.axes[0, :]:
    ax.set_xlabel(r"$t$")
    
p.figlegend.set_title("$c$")

plt.savefig("../figures/swe-bump-paper/nu-c-nlml-layout.pdf", dpi=600)

Now, we tabulate these results, first in a plot and then in a LaTeX table, to be plugged into the paper. Results confirm what the above plots suggest. Performance is best for the models which have the smaller viscosity parameters, and those with $c = 10$:

In [ ]:
import matplotlib.colors as colors

# mean the RMSEs, sum the LMLs
total_rmse = errors_nonlinear.isel(nt_skip=3).mean(dim="t").rmse_rel
total_lml = np.abs(errors_nonlinear.isel(nt_skip=3).lml).sum(dim="t")

rmse_array = total_rmse.to_numpy()[:, :]
lml_array = total_lml.to_numpy()[:, :]

fig, ax = plt.subplots(1, 2, 
                       figsize=(10, 4), constrained_layout=True,
                      sharey=True)
im = ax[0].pcolormesh(range(len(total_rmse.coords["c"].to_numpy())), 
                      range(len(total_rmse.coords["nu"].to_numpy())),
                      rmse_array,
                      norm=colors.LogNorm(vmin=rmse_array.min(), 
                                          vmax=rmse_array.max()))
ax[0].set_xticks(range(len(total_rmse.coords["c"].to_numpy())), 
                 total_rmse.coords["c"].to_numpy())
ax[0].set_yticks(range(len(total_rmse.coords["nu"].to_numpy())), 
                 total_rmse.coords["nu"].to_numpy())
ax[0].set_xlabel(r"$c$")
ax[0].set_ylabel(r"$\nu$")
ax[0].set_title("Average Rel. RMSE")
plt.colorbar(im, ax=ax[0])

im = ax[1].pcolormesh(range(len(total_lml.coords["c"].to_numpy())), 
                      range(len(total_lml.coords["nu"].to_numpy())),
                      lml_array,
                      norm=colors.LogNorm(vmin=lml_array.min(), 
                                          vmax=lml_array.max()))
ax[1].set_xticks(range(len(total_lml.coords["c"].to_numpy())), 
                 total_lml.coords["c"].to_numpy())
ax[1].set_yticks(range(len(total_lml.coords["nu"].to_numpy())), 
                 total_lml.coords["nu"].to_numpy())
ax[1].set_xlabel(r"$c$")
ax[1].set_title("Negative LML")
plt.colorbar(im, ax=ax[1])

plt.savefig("../figures/swe-bump-paper/nu-c-rrmse-lml-table-color.pdf", dpi=600)

In [ ]:
lml_formatted = np.apply_along_axis(format_col, 0, lml_array, True)

print(tabulate(lml_formatted, 
               headers=total_lml.coords['c'].to_numpy(),
               showindex=total_lml.coords['nu'].to_numpy()))

with open("../figures/swe-bump-paper/nu-c-lml-table.tex", "w") as f:
    f.write(tabulate(lml_formatted,
                     headers=total_lml.coords['c'].to_numpy(),
                     showindex=total_lml.coords['nu'].to_numpy(),
                     tablefmt="latex_raw"))

## Development plots: leftovers that we use to check models
Here, we plot the posterior. On the left, below, are the results for the linear model, and on the right are the results of the nonlinear model. Results are sensible with quite significant stratification across the $c$ values. We see that there is a reasonable drop in the relative errors in comparison to the prior:

In [ ]:
errors_prior.rmse_rel.interpolate_na(dim="t").isel(nu=[0, 1]).plot(
    hue="c", col="nu", sharey=False)

errors_nonlinear.rmse_rel.interpolate_na(dim="t").isel(nt_skip=3, nu=[0, 1]).plot(
    hue="c", col="nu", sharey=False, ylim=[0., 2.4])
plt.show()

Next, we plot these same results, but we instead use the log-marginal likelihood as the metric. The stratification here becomes more apparent across the board:

In [ ]:
np.abs(errors_nonlinear).lml.interpolate_na(dim="t").isel(nt_skip=3, nu=[0, 1]).plot(
    hue="c", col="nu", sharey=False, yscale="log")
plt.show()

And now for total comparison we show all of these results, for all the $\nu$ values:

In [ ]:
np.abs(errors_nonlinear.lml).isel(nt_skip=3).interpolate_na(dim="t").plot(
    hue="c", col="nu", sharey=True, yscale="log")
plt.show()

Next, we plot the results for the true value of $c$, for the various values of $\nu$. We see that there is a drastic change in the dynamics as we increase $\nu \geq 1$; once setting $\nu = 1$ this induces a degree of mismatch which results in the model not improving *a posteriori*:

In [ ]:
nu = [0, 1, 2, 3, 4]
fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(10, 4), sharey=True)
errors_prior.rmse_rel.isel(nu=nu, c=2).interpolate_na(dim="t").plot(
    hue="nu", ax=ax[0], add_legend=False)
ax[0].set_title("Prior, $c = 10$")
errors_nonlinear.rmse_rel.isel(nt_skip=3, nu=nu, c=2).interpolate_na(dim="t").plot(
    hue="nu", ax=ax[1], add_legend=True)
ax[1].set_title("Posterior, $c = 10$")
plt.show()

Next, we plot the log-marginal likelihoods for the true value $c = 10$, again stratified across the different $\nu$ values. In this case we see that there is a clear stratification as a result of the $\nu$.

In [ ]:
nu = nu
np.abs(errors_nonlinear.lml).isel(nt_skip=3, nu=nu, c=2).interpolate_na(dim="t").plot(
    hue="nu", add_legend=True, yscale="log")
plt.show()

In [ ]:
np.abs(errors_nonlinear.lml).isel(nt_skip=3, nu=nu).interpolate_na(dim="t").plot(
    hue="nu", col="c", sharey=True, yscale="log")
plt.show()

In [ ]:
import matplotlib.colors as colors

# mean the RMSEs, sum the LMLs
total_rmse = errors_nonlinear.isel(nt_skip=3).mean(dim="t").rmse_rel
total_lml = np.abs(errors_nonlinear.isel(nt_skip=3).lml).sum(dim="t")

rmse_array = total_rmse.to_numpy()[:, :]
lml_array = total_lml.to_numpy()[:, :]

fig, ax = plt.subplots(1, 2, figsize=(10, 4), constrained_layout=True)
im = ax[0].pcolormesh(range(len(total_rmse.coords["c"].to_numpy())), 
                      range(len(total_rmse.coords["nu"].to_numpy())),
                      rmse_array,
                      norm=colors.LogNorm(vmin=rmse_array.min(), 
                                          vmax=rmse_array.max()))
ax[0].set_xticks(range(len(total_rmse.coords["c"].to_numpy())), 
                 total_rmse.coords["c"].to_numpy())
ax[0].set_yticks(range(len(total_rmse.coords["nu"].to_numpy())), 
                 total_rmse.coords["nu"].to_numpy())
ax[0].set_title("Average Rel. RMSE")
plt.colorbar(im, ax=ax[0])

im = ax[1].pcolormesh(range(len(total_lml.coords["c"].to_numpy())), 
                      range(len(total_lml.coords["nu"].to_numpy())),
                      lml_array,
                      norm=colors.LogNorm(vmin=lml_array.min(), 
                                          vmax=lml_array.max()))
ax[1].set_xticks(range(len(total_lml.coords["c"].to_numpy())), 
                 total_lml.coords["c"].to_numpy())
ax[1].set_yticks(range(len(total_lml.coords["nu"].to_numpy())), 
                 total_lml.coords["nu"].to_numpy())
ax[1].set_title("Negative LML")
plt.colorbar(im, ax=ax[1])
plt.show()

In [ ]:
total_lml.plot(marker="o", col="nu", yscale="log", sharey=False)
plt.show()

In [ ]:
total_lml.plot(marker="o", col="c", xscale="symlog", yscale="log",sharey=False)
plt.show()

In [ ]:
print(tabulate(total_lml, 
               headers=total_lml.coords['c'].to_numpy(),
               showindex=total_lml.coords['nu'].to_numpy()))